<a href="https://colab.research.google.com/github/Viky98/AI-DL-MSDSP--458/blob/main/Sridhar_Vignesh_A_3_Using_LLMs_for_Entity_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Install Libraries

In [2]:
!pip install transformers torch  # For PyTorch

!pip install transformers tensorflow  # For TensorFlow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00


# 2. Download Spacy Model

In [3]:
import spacy

# Download Spacy English model
!python -m spacy download en_core_web_sm


2023-11-06 18:02:28.450007: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 18:02:28.450076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 18:02:28.450115: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 18:02:32.065340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# 3. Download Pre-trained Language Model

In [4]:
from transformers import BertTokenizer, BertModel

# Download pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [5]:
import pandas as pd
from transformers import BertTokenizer
# Connect to Drive
from google.colab import drive
drive.mount('/content/drive/')

# Load the data
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/Discussion/IMDB_Dataset.csv')

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create lists to store tokenized and encoded reviews
tokenized_reviews = []
encoded_reviews = []

# Tokenize and encode each review
for review in df['review']:
    # Tokenize the review
    tokens = tokenizer.tokenize(review)

    # Encode the tokens and add special tokens for [CLS] and [SEP]
    encoded = tokenizer.encode(
        tokens,
        add_special_tokens=True,
        max_length=512,  # You can adjust this based on your dataset
        truncation=True  # Truncate if the text is too long
    )

    # Append the tokenized and encoded review to the respective lists
    tokenized_reviews.append(tokens)
    encoded_reviews.append(encoded)

# Example: Access tokenized and encoded review for the first sample
print("Tokenized Review:", tokenized_reviews[0])
print("Encoded Review:", encoded_reviews[0])
# Select 25 random reviews
random_reviews = df.sample(n=25)['review'].tolist()

Mounted at /content/drive/
Tokenized Review: ['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', '1', 'oz', 'episode', 'you', "'", 'll', 'be', 'hooked', '.', 'they', 'are', 'right', ',', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'un', '##fl', '##in', '##ching', 'scenes', 'of', 'violence', ',', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', '.', 'trust', 'me', ',', 'this', 'is', 'not', 'a', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'tim', '##id', '.', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', ',', 'sex', 'or', 'violence', '.', 'its', 'is', 'hardcore', ',', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'gi

# 4: Preprocess Data (Tokenization and Encoding)

In [6]:
# Tokenize and encode text data using BERT tokenizer
tokenized_input = tokenizer(random_reviews, padding=True, truncation=True, return_tensors='pt')


# 5. Entity Extraction with Spacy

In [7]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

for review in random_reviews:
    entities = extract_entities(review)
    print("Entities:", entities)


Entities: [('first', 'ORDINAL'), ('UK', 'GPE'), ('1980', 'DATE'), ('Australian', 'NORP'), ("the Governor-Generals '", 'FAC'), ('first', 'ORDINAL'), ('Khemlani', 'PERSON'), ('Khemlani', 'ORG'), ('Labour', 'ORG'), ('the Governor-General', 'FAC'), ('Whitlam', 'ORG'), ('Kerr', 'ORG'), ('Commonwealth', 'ORG'), ('Carte Blanche', 'ORG')]
Entities: [('60', 'CARDINAL'), ('first', 'ORDINAL'), ('one', 'CARDINAL'), ('Joe', 'PERSON'), ('Joe Dallesandro', 'PERSON'), ('Warhol', 'PERSON'), ('Stinker', 'PRODUCT')]
Entities: [('5', 'CARDINAL'), ('Only like three', 'CARDINAL'), ('nine months', 'DATE'), ('Britney', 'ORG'), ('Federline(okay', 'PERSON'), ('Federline', 'ORG')]
Entities: [('Manos', 'NORP'), ('two', 'CARDINAL'), ('Crown International', 'ORG'), ('Crown', 'PRODUCT'), ('100', 'CARDINAL'), ('one', 'CARDINAL')]
Entities: [('Jacqueline McKenzie', 'PERSON'), ('John Lynch', 'PERSON'), ('two', 'CARDINAL')]
Entities: [('MOVIE', 'NORP'), ('EVER', 'ORG'), ('EVER', 'ORG'), ('EVER', 'ORG'), ('Tom Savini', '

# 6. Entity and Relation Extraction with LLM+Spacy

In [8]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
import spacy

# Load a pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tokenizer.vocab))

# Load a pre-trained spaCy model for entity recognition
nlp = spacy.load("en_core_web_sm")

# Define a function to tokenize and process the input text
def process_text(text, max_length=512):
    tokenized_input = tokenizer(text, max_length=max_length, truncation=True, padding=True, return_tensors='pt')
    return tokenized_input

# Process each review, truncate if longer than 512 tokens
processed_reviews = []
for review in random_reviews:
    processed_input = process_text(review)
    processed_reviews.append(processed_input)

# Perform entity extraction using pre-trained BERT model
entity_labels = []
with torch.no_grad():
    for input_batch in processed_reviews:
        outputs = model(**input_batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=2)
        entity_labels.extend(predictions.tolist())

# Example entity extraction using BERT + spaCy
for i, (review, entity_labels_batch) in enumerate(zip(random_reviews, entity_labels)):
    print(f"Review {i + 1}:")
    tokens = tokenizer.tokenize(review)
    entities = nlp(review).ents

    for j, (token, entity_label) in enumerate(zip(tokens, entity_labels_batch)):
        print(f"Token: {token}, Entity Label: {entity_label}, SpaCy Entity: {entities[j].text if j < len(entities) else 'N/A'}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Token: /, Entity Label: 3096, SpaCy Entity: N/A
Token: >, Entity Label: 21014, SpaCy Entity: N/A
Token: to, Entity Label: 25174, SpaCy Entity: N/A
Token: be, Entity Label: 11336, SpaCy Entity: N/A
Token: fair, Entity Label: 16414, SpaCy Entity: N/A
Token: ,, Entity Label: 8607, SpaCy Entity: N/A
Token: crown, Entity Label: 11926, SpaCy Entity: N/A
Token: international, Entity Label: 30002, SpaCy Entity: N/A
Token: really, Entity Label: 15840, SpaCy Entity: N/A
Token: destroyed, Entity Label: 24364, SpaCy Entity: N/A
Token: the, Entity Label: 1673, SpaCy Entity: N/A
Token: movie, Entity Label: 19938, SpaCy Entity: N/A
Token: by, Entity Label: 4193, SpaCy Entity: N/A
Token: adding, Entity Label: 24364, SpaCy Entity: N/A
Token: bad, Entity Label: 11998, SpaCy Entity: N/A
Token: music, Entity Label: 11998, SpaCy Entity: N/A
Token: and, Entity Label: 29656, SpaCy Entity: N/A
Token: doing, Entity Label: 6395, SpaCy Entity: N/A
Token: a, Enti

In [9]:
import pandas as pd
import re

# Define a list of entity keywords (words or phrases representing entities)
entity_keywords = ['actor', 'actress', 'director', 'producer', 'movie', 'character']

# Define a list of relation keywords (words or phrases representing relations between entities)
relation_keywords = ['acted in', 'directed', 'produced', 'starred', 'played the role of']

# Function to extract entities and relations based on keywords
def extract_entities_relations(text):
    entities = []
    relations = []
    for keyword in entity_keywords:
        entity_matches = re.findall(r'\b{}\b'.format(keyword), text, flags=re.IGNORECASE)
        entities.extend(entity_matches)
    for keyword in relation_keywords:
        relation_matches = re.findall(r'\b{}\b'.format(keyword), text, flags=re.IGNORECASE)
        relations.extend(relation_matches)
    return list(set(entities)), list(set(relations))

# Apply the function to the dataset
ground_truth_entities = []
ground_truth_relations = []

for review in df['review']:
    entities, relations = extract_entities_relations(review)
    ground_truth_entities.append(entities)
    ground_truth_relations.append(relations)

# Add the generated ground truth labels to the DataFrame
df['ground_truth_entities'] = ground_truth_entities
df['ground_truth_relations'] = ground_truth_relations

# Save the DataFrame with ground truth labels
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/Discussion/IMDB_Dataset_with_ground_truth.csv', index=False)


# 7: Evaluation of Performance

In [12]:
import pandas as pd

# Load the dataset with ground truth labels
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/Discussion/IMDB_Dataset_with_ground_truth.csv')

# Function to calculate precision, recall, and F1-score
def calculate_metrics(true_positives, false_positives, false_negatives):
    precision = true_positives / (true_positives + false_positives + 1e-9)
    recall = true_positives / (true_positives + false_negatives + 1e-9)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
    return precision, recall, f1_score

# Calculate metrics for entity extraction
entity_true_positives = 0
entity_false_positives = 0
entity_false_negatives = 0


# Calculate predicted_entities and predicted_relations based on your models

predicted_entities = []
predicted_relations = []

for review in df['review']:
    entities, relations = extract_entities_relations(review)
    predicted_entities.append(entities)
    predicted_relations.append(relations)

# Add the predicted_entities and predicted_relations to your DataFrame
df['predicted_entities'] = predicted_entities
df['predicted_relations'] = predicted_relations


for i, (predicted_entities, true_entities) in enumerate(zip(df['predicted_entities'], df['ground_truth_entities'])):
    for entity in predicted_entities:
        if entity in true_entities:
            entity_true_positives += 1
        else:
            entity_false_positives += 1
    entity_false_negatives += len(true_entities) - len(predicted_entities)

entity_precision, entity_recall, entity_f1_score = calculate_metrics(entity_true_positives, entity_false_positives, entity_false_negatives)

# Calculate metrics for relation extraction
relation_true_positives = 0
relation_false_positives = 0
relation_false_negatives = 0

for i, (predicted_relations, true_relations) in enumerate(zip(df['predicted_relations'], df['ground_truth_relations'])):
    for relation in predicted_relations:
        if relation in true_relations:
            relation_true_positives += 1
        else:
            relation_false_positives += 1
    relation_false_negatives += len(true_relations) - len(predicted_relations)

relation_precision, relation_recall, relation_f1_score = calculate_metrics(relation_true_positives, relation_false_positives, relation_false_negatives)

print("Entity Extraction Metrics:")
print("Precision:", entity_precision)
print("Recall:", entity_recall)
print("F1-score:", entity_f1_score)

print("Relation Extraction Metrics:")
print("Precision:", relation_precision)
print("Recall:", relation_recall)
print("F1-score:", relation_f1_score)


Entity Extraction Metrics:
Precision: 0.9999999999999819
Recall: 0.09362778496968029
F1-score: 0.1712242248612287
Relation Extraction Metrics:
Precision: 0.9999999999997288
Recall: 0.026863975168855955
F1-score: 0.05232236360863105
